In [1]:
from malaya_speech.utils import torch_featurization
import random
import torch
import malaya_speech
from malaya_speech.utils.char import HF_CTC_VOCAB
from conformer import ConformerConfig, ConformerEncoder

HF_CTC_VOCAB = HF_CTC_VOCAB + ['_']
HF_CTC_VOCAB_INDEX = {no: c for no, c in enumerate(HF_CTC_VOCAB)}
HF_CTC_VOCAB_REV = {v: k for k, v in HF_CTC_VOCAB_INDEX.items()}

ConformerConfig.register_for_auto_class()
ConformerEncoder.register_for_auto_class()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/.local/lib/python3.8/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [2]:
config = ConformerConfig(
    input_dim=80,
    output_dim=len(HF_CTC_VOCAB),
    time_reduction_stride=4,
    conformer_input_dim=144,
    conformer_ffn_dim=576,
    conformer_num_layers=8,
    conformer_num_heads=4,
    conformer_depthwise_conv_kernel_size=31,
    conformer_dropout=0.1,
    pad_token_id=len(HF_CTC_VOCAB) - 1,
    ctc_loss_reduction='mean',
    ctc_zero_infinity=True,
)

In [3]:
encoder = ConformerEncoder(config)

In [4]:
global_stats = torch_featurization.GlobalStatsNormalization('../../../malay-stats.json')

In [5]:
y, sr = malaya_speech.load('../../../speech/example-speaker/husein-zolkepli.wav')
y.shape[0] / sr

5.630625

In [6]:
srs = [4400, 5100, 6000, 8000, 10000]

def downsample(y, sr):
    s_sr = random.choice(srs)
    y_ = malaya_speech.resample(y, sr, s_sr)
    return malaya_speech.resample(y_, s_sr, sr)

In [7]:
mel = torch_featurization.melspectrogram(y)
mel = torch_featurization.piecewise_linear_log(mel)

In [8]:
%%time

encoder(mel.unsqueeze(0), torch.tensor([mel.shape[0]]))

CPU times: user 54.8 s, sys: 153 ms, total: 54.9 s
Wall time: 7.58 s


(tensor([[[-0.3420, -0.5635, -0.2499,  ...,  0.0857,  0.6430, -0.1187],
          [ 0.0123, -0.7919, -0.1585,  ..., -0.0232,  0.5699, -0.3159],
          [-0.1957, -0.6225, -0.4423,  ..., -0.2093,  0.5401, -0.1917],
          ...,
          [-0.2118, -1.2328, -0.1823,  ...,  0.3269,  0.0499,  0.1977],
          [-0.2337, -1.3522,  0.0487,  ..., -0.0912,  0.4360, -0.2936],
          [-0.4794, -1.4853, -0.2047,  ..., -0.0595,  0.4742,  0.2245]]],
        grad_fn=<ViewBackward0>),
 tensor([141]))

In [9]:
text = ['nama saya husein bin zolkepli']
text = [[HF_CTC_VOCAB_REV[c] for c in t] for t in text]
labels = torch.tensor(text)

In [11]:
encoder(mel.unsqueeze(0), torch.tensor([mel.shape[0]]), labels)

(tensor(14.1196, grad_fn=<MeanBackward0>),
 tensor([[[ 0.1176, -0.6080, -0.3661,  ..., -0.2499,  0.4605, -0.5665],
          [-0.2955, -1.0030, -0.4919,  ...,  0.0818,  0.6888, -0.5607],
          [-0.0176, -0.4774, -0.3897,  ...,  0.0675,  0.8695, -0.3563],
          ...,
          [ 0.1790, -1.6942,  0.2734,  ..., -0.0593,  0.9315, -0.1103],
          [-0.1625, -1.6541, -0.0828,  ..., -0.1314,  0.3964, -0.3435],
          [-0.2599, -1.7252,  0.2109,  ..., -0.1284,  0.0713, -0.3158]]],
        grad_fn=<ViewBackward0>),
 tensor([141]))

In [12]:
total_params = sum(
    param.numel() for param in encoder.parameters()
)
total_params

3937720

In [ ]:
encoder.save_pretrained('./out')

In [ ]:
!ls -lh out

In [ ]:
encoder.push_to_hub('huseinzol05/conformer-tiny', safe_serialization = True)

In [ ]:
from transformers import AutoConfig, AutoModel

In [ ]:
model = AutoModel.from_pretrained('huseinzol05/conformer-tiny', trust_remote_code = True)